In [1]:
from creating_classes import creating_classes

In [2]:
creating_classes('test', 6)

1/1 [==============================] - 0s 160ms/step


ValueError: Found array with dim 5. PCA expected <= 2.